# 04 Vector DataBase

## 1. Load Libraries and Data

In [3]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http import models
from datasets import load_dataset, Audio
import numpy as np

hugggof/music-caption-eval-v2

In [6]:
dataset = load_dataset(
    "audiofolder", 
    data_dir="../data/Audios/", 
    # drop_metadata=True, 
    # split="train"
)
dataset

Resolving data files:   0%|          | 0/1500 [00:00<?, ?it/s]

Found cached dataset audiofolder (/home/ramonperez/.cache/huggingface/datasets/audiofolder/default-937056b5ef3f06eb/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 1500
    })
})

In [ ]:
vectors = np.load('vectors.npy')
vectors.shape

(1500, 768)

In [ ]:
client = QdrantClient("localhost", port=6333)

In [ ]:
client.recreate_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [ ]:
from pprint import pprint
collection_info = client.get_collection(collection_name="test_collection")
collection_info

method

In [ ]:
from qdrant_client.http.models import CollectionStatus

assert collection_info.status == CollectionStatus.GREEN
assert collection_info.vectors_count == 0

In [ ]:
len(vectors)

1500

In [ ]:
client.upsert(
    collection_name="test_collection",
    points=models.Batch(
        ids=list(range(len(vectors))),
        vectors=vectors.tolist()
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
from diffusers import AudioLDMPipeline
import torch

repo_id = "cvssp/audioldm"
pipe = AudioLDMPipeline.from_pretrained(repo_id)
pipe = pipe.to("cuda")

prompt = "high quality bachata"

audio = pipe(prompt=prompt, num_inference_steps=500, audio_length_in_s=5.0).audios[0]

from IPython.display import Audio

Audio(audio, rate=16000)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
classifier(audio, ).

[{'score': 0.356492280960083, 'label': 'Salsa'},
 {'score': 0.17768427729606628, 'label': 'Merengue'},
 {'score': 0.16411282122135162, 'label': 'Cumbia'},
 {'score': 0.156268909573555, 'label': 'Vallenato'},
 {'score': 0.14544172585010529, 'label': 'Bachata'}]

In [ ]:
audio.shape

(80000,)

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("first_mod")
inputs = feature_extractor(audio, sampling_rate=feature_extractor.sampling_rate, return_tensors="pt", max_length=16000, truncation=True)

In [ ]:
inputs['input_values'].size()

torch.Size([1, 16000])

In [ ]:
with torch.no_grad():
    last_hidden_state = model(**inputs.to(device)).last_hidden_state[:, 0]
last_hidden_state.size()

torch.Size([1, 768])

In [ ]:
vectr = last_hidden_state.cpu().numpy()[0, :]

In [ ]:
results2 = client.search(
    collection_name="test_collection",
    query_vector=vectr,
    limit=10, 
    # with_vectors=True
)
results2

[ScoredPoint(id=758, version=0, score=0.84251463, payload={}, vector=None),
 ScoredPoint(id=634, version=0, score=0.8059299, payload={}, vector=None),
 ScoredPoint(id=687, version=0, score=0.7902696, payload={}, vector=None),
 ScoredPoint(id=635, version=0, score=0.7857487, payload={}, vector=None),
 ScoredPoint(id=642, version=0, score=0.7841639, payload={}, vector=None),
 ScoredPoint(id=637, version=0, score=0.7818465, payload={}, vector=None),
 ScoredPoint(id=683, version=0, score=0.76815987, payload={}, vector=None),
 ScoredPoint(id=721, version=0, score=0.7639689, payload={}, vector=None),
 ScoredPoint(id=663, version=0, score=0.75371236, payload={}, vector=None),
 ScoredPoint(id=601, version=0, score=0.7534412, payload={}, vector=None)]

In [ ]:
one_array = np.array(results[0].dict()["vector"])

In [ ]:
music = []

for result in results:
    the_song = Audio(np.array(result.dict()["vector"]), rate=16_000)
    # feature_extractor(the_song, sampling_rate=feature_extractor.sampling_rate, return_tensors="pt", max_length=16000, truncation=True)
    music.append(the_song)

In [ ]:
music[2]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
np.sum(one_array)

-0.3755155722250001

In [ ]:
np.sum(vectors, axis=0) == np.sum(one_array)

0

In [ ]:
scores = cosine_similarity([one_array], vectors)[0]
scores

array([-0.25646476, -0.2657308 ,  0.03965592, ..., -0.48054826,
       -0.33845106, -0.40019513])

In [ ]:
top_scores_ids = np.argsort(scores)[-5:][::-1]
top_scores_ids

array([758, 634, 635, 890, 615])